In [1]:
import pandas as pd

In [2]:
# Loading the dataset

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[: ,3:13]
y = dataset.iloc[:,13]

In [3]:
# Loading categorical variables
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

# Concating categorical Variables
X = pd.concat([X, gender, geography], axis=1)

# Removing unnecessary data
X=X.drop(['Geography', 'Gender'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train )
X_test = sc.transform(X_test)        

In [5]:
# Hyperparameter Optimization

import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [6]:
# For Grid Search

def create_model(layers, activation, optimizer):
    # Initialing the model
    model = Sequential()
    
    for i, nodes in enumerate(layers):
        if(i==0):
            model.add(Dense(nodes, activation=activation, input_dim=X_train.shape[1])) #For first input layer
        else: 
            model.add(Dense(nodes, activation=activation)) # For hidden layers
        # Common for input and hidden layers    
        # model.add(Activation(activation))
        # model.add(Dropout(0.3))
        
    # Output Layer
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
        
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# KerasClassifier is a wrapper of k-fold Cross Validation
model = KerasClassifier(build_fn=create_model, verbose=2)

In [13]:
# defining parameters for Grid Search
# layers = [(6,6)]
# activations = [keras.activations.relu]
# optimizers = [keras.optimizers.Adam]
layers = [(20,), (40,20), (45,30,15),(6,6)]
activations = ['sigmoid', 'relu']
optimizers = [keras.optimizers.Adam(), keras.optimizers.RMSprop()]
# optimizers = ['adam', 'rmsprop']
param_grid = dict(
    layers=layers, 
    activation=activations, 
    optimizer = optimizers, 
    # batch_size=[10], 
    # epochs=[30]
    batch_size=[128, 256, 25, 32], 
    epochs=[30,100,500]
    )

In [14]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring = 'accuracy', cv=10)

In [15]:
# Running the grid model
grid_result = grid.fit(X_train, y_train)


0/50 - 0s - loss: 0.4761 - accuracy: 0.7921
Epoch 4/30
50/50 - 0s - loss: 0.4633 - accuracy: 0.7922
Epoch 5/30
50/50 - 0s - loss: 0.4532 - accuracy: 0.7940
Epoch 6/30
50/50 - 0s - loss: 0.4458 - accuracy: 0.7981
Epoch 7/30
50/50 - 0s - loss: 0.4403 - accuracy: 0.8030
Epoch 8/30
50/50 - 0s - loss: 0.4366 - accuracy: 0.8060
Epoch 9/30
50/50 - 0s - loss: 0.4342 - accuracy: 0.8063
Epoch 10/30
50/50 - 0s - loss: 0.4325 - accuracy: 0.8092
Epoch 11/30
50/50 - 0s - loss: 0.4312 - accuracy: 0.8094
Epoch 12/30
50/50 - 0s - loss: 0.4301 - accuracy: 0.8097
Epoch 13/30
50/50 - 0s - loss: 0.4293 - accuracy: 0.8100
Epoch 14/30
50/50 - 0s - loss: 0.4284 - accuracy: 0.8102
Epoch 15/30
50/50 - 0s - loss: 0.4275 - accuracy: 0.8098
Epoch 16/30
50/50 - 0s - loss: 0.4266 - accuracy: 0.8100
Epoch 17/30
50/50 - 0s - loss: 0.4257 - accuracy: 0.8111
Epoch 18/30
50/50 - 0s - loss: 0.4246 - accuracy: 0.8108
Epoch 19/30
50/50 - 0s - loss: 0.4237 - accuracy: 0.8122
Epoch 20/30
50/50 - 0s - loss: 0.4227 - accuracy: 

KeyboardInterrupt: 

In [16]:
# Printing the grid result
[grid_result.best_score_,grid_result.best_params_]

NameError: name 'grid_result' is not defined

In [ ]:
# Using Cross Validation Score
# def build_classifier():
#     classifier = Sequential()
#     classifier.add(Dense(units=6, activation="relu", kernel_initializer="he_uniform", input_dim = 11))
#     classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
#     classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))
#     classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#     return classifier    

# model = KerasClassifier(build_fn=build_classifier, batch_size = 10, epochs=100)
# cvs = cross_val_score(estimator= model, X=X_train, y=y_train, cv=10, verbose=2)